In [1]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

In [2]:
%matplotlib widget

### Discharge data

In [26]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 10
drive_freq = 71
fsamp = 5000
def func1_scale(i):
    
    bb = step_cal_files[i]
        
#     freq2 = np.fft.rfftfreq(len(bb.electrode_data[0]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.electrode_data[0]))
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp2,np.std(bb.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1000, 1000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [5]:
step_cal_dir = r'/data/new_trap/20210830/Bead1/Discharge/FineDischarge/'
step_cal_files = load_dir(step_cal_dir, start_file=0)

100%|██████████| 185/185 [00:04<00:00, 39.94it/s]

185  files in folder
185  files loaded


In [22]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 185
[0.07216374] 0.05004696352452862
1 / 185
[0.07218011] 0.05004674261804184
2 / 185
[0.07217364] 0.05004664610410522
3 / 185
[0.07219365] 0.05004697284574258
4 / 185
[0.07219079] 0.050046528696682864
5 / 185
[0.07219249] 0.050046733284185005
6 / 185
[0.07221319] 0.05004681596322339
7 / 185
[0.07223124] 0.05004672905591215
8 / 185
[0.07222904] 0.05004656813982753
9 / 185
[0.07222753] 0.05004692924467627
10 / 185
[0.07222463] 0.050046983363467364
11 / 185
[0.07225377] 0.050046594234892855
12 / 185
[0.07225198] 0.050046846698829875
13 / 185
[0.07223048] 0.05004707515321758
14 / 185
[0.07223096] 0.05004684113100025
15 / 185
[0.07213118] 0.050046489889492105
16 / 185
[0.0721514] 0.05004654773486393
17 / 185
[0.07216683] 0.05004690868597997
18 / 185
[0.07216085] 0.050046393547231255
19 / 185
[0.07216514] 0.05004671808595284
20 / 185
[0.07216004] 0.05004681631103741
21 / 185
[0.07215716] 0.050046393634943384
22 / 185
[0.07216653] 0.05004636430981544
23 / 185
[0.07217415] 0.0500467585547

In [40]:
scale = np.abs(np.mean(amps3[120:125])/5)
print("Scale is ", scale)
bb = step_cal_files[-1]
voltage = np.std(bb.electrode_data[0])*np.sqrt(2)*100
print("voltage: ", voltage)

Scale is  4.17335521797213e-05
voltage:  5.004728564882627


In [41]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots()
# scale = -3.54295075496134e-05

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')

ax.set(xlabel='dataset', ylabel='Electron charges', ylim=(-3,6.5), xlim=(117,185))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 10
drive_freq = 71

def get_amp(i, foldername):
    
    fname =foldername+'/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    drive_freq = 71+1
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 'sigma': 0.1,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 'limit_sigma': [0, 1],
                  'limit_A': [-100, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = 142+1
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': np.pi/2, 'sigma': 0.1,
              'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
              'limit_phi': [-2 * np.pi, 2 * np.pi], 'limit_sigma': [0, 1],
              'limit_A': [-100, 100], 
              'print_level': 0, 'fix_f': True, 'fix_phi': True}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], m1_tmp.values[3], m2_tmp.values[0]

In [29]:
i=0
foldername = r'/data/new_trap/20210830/Bead1/Neutrality/20V_100V_YZspin_98p98kHz_20210901/'
get_amp(i, foldername=foldername)/scale

array([-0.03803174,  8.7415255 ,  0.04518386])

In [37]:
i = 4
fname = r'/data/new_trap/20210830/Bead1/Neutrality/20V_100V_YZspin_98p98kHz_20210901/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
voltage0_20 = np.std(bb.electrode_data[0])*np.sqrt(2)*100
voltage1_20 = np.std(bb.electrode_data[1])*np.sqrt(2)*100
bias2 = np.mean(bb.electrode_data[0])*100
print("voltages, bias: ", voltage0_20, voltage1_20)

voltages, bias:  10.004629671441691 0.019457077173451576


In [42]:
voltage, voltage0_20

(5.004728564882627, 10.004629671441691)

In [43]:
foldername = r'/data/new_trap/20210830/Bead1/Neutrality/20V_100V_YZspin_98p98kHz_20210901/'
amps3_data_20V_1 = np.array(Parallel(n_jobs=40)(delayed(get_amp)(i, foldername) for i in tqdm(range(7600))))

100%|██████████| 7600/7600 [01:38<00:00, 76.59it/s] 


In [44]:
base_folder =  r'/data/new_trap/20210830/Bead1/Neutrality'
fnames = [base_folder+'/20V_100V_YZspin_98p98kHz_20210901/',
          base_folder+'/20V_100V_YZspin_98p98kHz_20210903/',
          base_folder+'/20V_100V_YZspin_98p98kHz_20210904/',
          base_folder+'/20V_100V_YZspin_98p98kHz_20210905/']

files = [7600, 1380, 7220, 3700]

In [69]:
ams_neutrality_noise = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality_noise += list(np.array(amps_tmp))
ams_neutrality_noise = np.array(ams_neutrality_noise)

100%|██████████| 3700/3700 [00:36<00:00, 102.53it/s]


### Plot raw data

In [46]:
##
i=0; j=1
_,ax = plt.subplots()
dat = ams_neutrality
ax.scatter(dat[::4,i], dat[::4,j],label='positive electrode')
ax.scatter(dat[2::4,i], dat[2::4,j], label='negative electrode')
ax.set(xlabel='amplitude', ylabel='noise parameter')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
data0 = ams_neutrality[:,0]/scale*voltage/voltage0_20
factor = 1
dataPlus=np.array(list(data0[0::4]))
dataMinus=np.array(list(data0[2::4]))
data_A = dataPlus+factor*dataMinus

data0_noise = ams_neutrality_noise[:,0]/scale*voltage/voltage0_20
dataPlus_noise=np.array(list(data0_noise[0::4]))
dataMinus_noise=np.array(list(data0_noise[2::4]))
data_A_noise = dataPlus_noise+factor*dataMinus_noise


_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(dataPlus)), dataPlus, label='Y-axis/, plus, 200V, XZ-spin-46kHz, with-height-fb')
ax.scatter(np.arange(len(dataMinus)), dataMinus, label='Y-axis, minus, 200V, XZ-spin-46kHz, with-height-fb')
ax.plot(np.arange(0,len(dataPlus)), np.arange(0,len(dataPlus))*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean: ', np.mean(dataPlus), 'std: ', np.std(dataPlus)/np.sqrt(len(dataPlus)))
print('mean: ', np.mean(dataMinus), 'std: ', np.std(dataMinus)/np.sqrt(len(dataMinus)))
print(np.mean(data_A), np.std(data_A/np.sqrt(len(data_A))))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  0.00018623256679479842 std:  0.0004494098180695245
mean:  0.0001412413331960206 std:  0.0004213356325507815
0.0003274738999908192 0.0006116450053981934


In [77]:
N = len(dataMinus)

data_shifted = np.array([(dataMinus[i]+dataMinus[i-1])/2 for i in range(1,N)])
data3_A_shifted = data_shifted+factor*dataPlus[:-1]

data_shifted_noise = np.array([(dataMinus_noise[i]+dataMinus_noise[i-1])/2 for i in range(1,N)])
data3_A_shifted_noise = data_shifted_noise+factor*dataPlus_noise[:-1]

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data_A))*2*10/3600, data_A, label='20Vpp-z-plus, new trap')
ax.scatter(np.arange(len(data3_A_shifted))*2*10/3600, data3_A_shifted, label='20Vpp-z, A parameter corrected', alpha=0.6)

ax.plot(np.arange(len(data_A))*2*10/3600, np.arange(len(data_A))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, A parameter: mean, std: ', np.mean(data_A), np.std(data_A)/np.sqrt(len(data_A)))
print('1st harmonic, A-shifted: mean, std: ', np.mean(data3_A_shifted), np.std(data3_A_shifted)/np.sqrt(len(data3_A_shifted)))


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, A parameter: mean, std:  0.0003274738999908192 0.0006116450053981932
1st harmonic, A-shifted: mean, std:  0.0003246082888475219 0.0005396854659370785


In [78]:
len(dataPlus)*2*10/3600

27.63888888888889

In [74]:
epsilon_array = np.array([data3_A_shifted, data3_A_shifted_noise])
epsilon_array.shape

(2, 4974)

In [75]:
np.save("epslion_new_trap_bead_20210830", epsilon_array)

In [65]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

data_hist = dataPlus+factor*dataMinus
range_pram = 0.1
data_hist = np.array(list(data3_A_shifted))
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data_hist, bins=31, range=(-range_pram, range_pram), label='200V-x, xz-spin-100V')
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]
# bins_fit = bins[:-1]
for i,ee in enumerate([entries]):
    x = bins_fit
    y= ee

#     n = len(x)                          #the number of data
#     mean = float(0)                  #note this correction
#     sigma = sum(y*(x-mean)**2)/n        #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[400,0,0.1])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], "cov: ", np.sqrt(pcov)[1,1])
    print("mean: ", popt[2]/np.sqrt(len(data_hist)), "std: ", popt[2])

mean:  0.00039284539400276223 cov:  0.0004496253230221058
mean:  -0.00045357986882582613 std:  -0.031989441772056024


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in sqrt


In [67]:
_,ax = plt.subplots()
ax.errorbar(bins_fit, entries, yerr=np.sqrt(entries), fmt='o', label='data')
ax.plot(x,gaus(x,*popt),'r--',label='fit')
ax.legend()
print("mean: ", popt[1], "std: ", np.sqrt(pcov)[1,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  0.00039284539400276223 std:  0.0004496253230221058


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in sqrt
  """
